In [1]:
%run /Users/simonhvidtfeldt/Group_project/Setup.py
# Det her kan i ignore eller lade være med at køre. Det er bare så jeg er sikker på at have installeret alt der skal bruges

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonhvidtfeldt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simonhvidtfeldt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Setup complete. All modules are ready.


### Task 2

***Consider whether it would make sense to include meta-data features as well. If so, which ones, and why? If relevant, report the performance when including these additional features and compare it to the first baselines. Discuss whether these results match your expectations.***

In Task 1 we were asked to train a simple logistic regression classifier using a fixed vocabulary of the 10.000 most frequently used words, but we have access to many different meta-data features as well. These are 

-   id
-   domain
-   type
-   url
-   content
-   scraped_at
-   inserted_at
-   updated_at
-   title
-   authors
-   keywords
-   meta_keywords
-   meta_description
-   tags
-   summary

Some of these are more interessting then others. "url" and "id" isnt very interesting in our simple logistic regression classifier. The "url" may include the domain, but the this can also be extracted from the "domain" feature. So including the actual "url" will just introduce unneseccary noice. Additionally, the "id" is just a identifier without any informative value. 

"scraped_at", "inserted_at" and "updated_at" are also not very interesting in it self, but could prove useful if we want to pull the timepatterens from the data.

"authors" is a bit tricky. If a author is known to publish fake news, then this could be a good indicator. But if we have a lot of different authors we just risk overfitting our model, therefor we want include it. 

Now, we get to the interesting parts. "domain" will most likely have a pattern. Some domains are more likely to publish fake-news then others, and a simple logistic regression classifier could be trained to predict this. Another interesting mete-feature is the "title". A title will often include words that are indicative of the content of the article, and many click-bait titles are often use the same pattern with a catching title. Whith this being said, we actually cleaned out this during cleaning, so we wont be able to extract any reliable data.

Similary to "title" is "meta_description" and "summary". These are often like the titles, but include more information about the content of the article.

"keywords" and "meta_keywords" can be very informative. These will typically include good signals about the content of the article, and will prove usefull in our classifier.

Lastly, "tags" is similar to "keywords" and "meta_keywords", and are therefor in a similar position. They will include useful information regarding the article, and is therefor also very useful for our classifier.

So we will try to include the following meta-data features:

-   meta_description
-   content
-   keywords
-   meta_keywords
-   tags
-   title

In [2]:
"""""""""""""""""""""""
CODE FROM PART_2_TASK_1
"""""""""""""""""""""""

import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

Final_file_path = "/Users/simonhvidtfeldt/Group_project/cleaned_file.csv"
data = pd.read_csv(Final_file_path, low_memory=False)

In [10]:
"""""""""""""""""""""""""""""""""""""""""""""
CODE FROM PART_2_TASK_1 *EDITED*
"""""""""""""""""""""""""""""""""""""""""""""

# Define mapping
label_mapping = {
    "['polit']": "True",
    "['clickbait']": "True",
    "['reliabl']": "True",
    "['fake']": "Fake",
    "['bia']": "Fake"
}

# Create the 'label' column
data["label"] = data["type"].map(label_mapping)
print(" - Created the 'label' column")

# Convert 'True' to 1 and 'Fake' to 0
data["label"] = data["label"].map({"True": 1, "Fake": 0})
print(" - Converted 'True' to 1 and 'Fake' to 0")

# Drop rows with NaN in the 'label' column (if any)
data = data.dropna(subset=["label"])
print(" - Dropped rows with NaN in the 'label' column")

# Save the DataFrame to a CSV file (if needed)
data.to_csv("cleaned_file_with_label.csv", index=False)
print(" - Saved the DataFrame to a CSV file")

Created the 'label' column
Converted 'True' to 1 and 'Fake' to 0
Dropped rows with NaN in the 'label' column
Saved the DataFrame to a CSV file


In [14]:
"""""""""""""""""""""""""""""""""""""""""""""
CODE FROM PART_2_TASK_1 *EDITED*

LOGISTIC REGRESSION FOR CONTENT 
"""""""""""""""""""""""""""""""""""""""""""""

print("Beginning Logistic regression for content:")

# Load CSV
data = pd.read_csv("/Users/simonhvidtfeldt/Group_project/cleaned_file_with_label.csv")
print("  - Loaded CSV.")

# Ensure text data is string type
data["content"] = data["content"].fillna("").astype(str)
print("  - Ensured text data is string type.")

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=10000, binary=True)
X = vectorizer.fit_transform(data["content"])
print("  - Vectorized text data.")

# Use the binary 'label' column as the target variable
y = data["label"].astype(int)  # Ensure 'label' is integer type
print("  - Used the binary 'label' column as the target variable.")

# Splitting dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print("  - Splitting dataset into train, validation, and test sets.")

# Implementing logistic regression
clf = LogisticRegression(max_iter=500, random_state=42, solver="saga", n_jobs=-1)
clf.fit(X_train, y_train)
print("  - Implementing logistic regression.")

# Evaluating the model
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average="binary")  # Use 'binary' for binary classification

# Display results
print("\nResults of logistic regression:")
print(f"  - F1-score: {f1:.4f}\n")
print("Hyperparameters for Logistic Regression:")
print(f"  - Max Iterations: {clf.max_iter}")
print(f"  - Solver: {clf.solver}")
print(f"  - Random State: {clf.random_state}")


Beginning Logistic regression:
    Loaded CSV.
    Ensured text data is string type.
    Vectorized text data.
    Used the binary 'label' column as the target variable.
    Splitting dataset into train, validation, and test sets.
    Implementing logistic regression.

Results of logistic regression:
    F1-score: 0.9194

Hyperparameters for Logistic Regression:
    Max Iterations: 500
    Solver: saga
    Random State: 42


In [16]:
"""""""""""""""""""""""""""""""""""""""""""""
CODE FROM PART_2_TASK_1 EDITED

LOGISTIC REGRESSION FOR CONTENT + META DATA
"""""""""""""""""""""""""""""""""""""""""""""

print("Beginning Logistic Regression with meta data features:")

# Load CSV
data = pd.read_csv("/Users/simonhvidtfeldt/Group_project/cleaned_file_with_label.csv")
print("  - Loaded CSV.")

# Ensure text data is string type and handle missing values
text_columns = ["content", "title", "meta_keywords", "meta_description", "tags"]
for col in text_columns:
    data[col] = data[col].fillna("").astype(str)
print("  - Ensured text data is string type and handled missing values.")

# Combine all text columns into a single feature
data["combined_text"] = (
    data["content"] + " " +
    data["title"] + " " +
    data["meta_keywords"] + " " +
    data["meta_description"] + " " +
    data["tags"]
)
print("  - Combined all text columns into a single feature.")

# Vectorize the combined text feature
vectorizer = TfidfVectorizer(max_features=10000, binary=True)
X = vectorizer.fit_transform(data["combined_text"])
print("  - Vectorized the combined text feature.")

# Use the binary 'label' column as the target variable
y = data["label"].astype(int)  # Ensure 'label' is integer type
print("  - Used the binary 'label' column as the target variable.")

# Splitting dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print("  - Splitting dataset into train, validation, and test sets.")

# Implementing logistic regression
clf = LogisticRegression(max_iter=500, random_state=42, solver="saga", n_jobs=-1)
clf.fit(X_train, y_train)
print("  - Implementing logistic regression.")

# Evaluating the model
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average="binary")  # Use 'binary' for binary classification

# Display results
print("\nResults of Logistic Regression with meta data features:")
print(f"  - F1-score: {f1:.4f}\n")
print("Hyperparameters for Logistic Regression:")
print(f"  - Max Iterations: {clf.max_iter}")
print(f"  - Solver: {clf.solver}")
print(f"  - Random State: {clf.random_state}")

Beginning Logistic Regression with meta data features:
  - Loaded CSV.
  - Ensured text data is string type and handled missing values.
  - Combined all text columns into a single feature.
  - Vectorized the combined text feature.
  - Used the binary 'label' column as the target variable.
  - Splitting dataset into train, validation, and test sets.
  - Implementing logistic regression.

Results of Logistic Regression with Additional Columns:
  - F1-score: 0.9296

Hyperparameters for Logistic Regression:
  - Max Iterations: 500
  - Solver: saga
  - Random State: 42


After adding the additional meta data features we where able to improve our F1 score from 0.9194 to 0.9296. This is as expected. By adding these additional features we are able to capture more information about the data and therefor improve the accuracy of our model. 

### Task 3

***Apply your data preprocessing pipeline to the extra reliable data you scraped during Graded Exercise 2 and add this to the training data and observe how this changes the performance of your simple model. Discuss whether you will continue to use this extra reliable data for the Advanced Model.***

***For the remainder of the project, we will limit ourselves to main-text data only (i.e. no meta-data). This makes it easier to do the cross-domain experiment in Part 4 (which does not have the same set of meta-data fields).***

In this task we apply our data preprocessing pipeline to the extra reliable data we scraped during Graded Exercise 2 and add this to the training data and observe how this changes the performance of our simple model.

In [18]:
import csv

def convert_to_csv(input_file, output_file):
    # Opens a text-file and reads all lines into a list
    with open(input_file, 'r') as f:
        lines = f.readlines()

    # Initializes a list to save articles and a dictionary to save metadata for the "current article"
    articles = []
    current_article = {}
    current_key = None

    # Checks each line. If empty, it means we've finished an article and the current article is added to articles
    for line in lines:
        line = line.strip()
        if not line:
            if current_article:
                articles.append(current_article)
                current_article = {}
                current_key = None
        else:
            # Check if line contains a ":". If it does we create a key and a value. Example "headline: "Some headline".
            # If it doesn't we just add the line to the current article
            if ':' in line:
                key, value = line.split(':', 1)
                key = key.strip()
                value = value.strip()
                current_article[key] = value
                current_key = key
            else:
                if current_key and current_key in current_article:
                    current_article[current_key] += ' ' + line

    # Adds the last article to the list "articles"
    if current_article:
        articles.append(current_article)

    # Defines the csv columm names
    headers = [
        '', 'id', 'domain', 'type', 'url', 'content', 'scraped_at',
        'inserted_at', 'updated_at', 'title', 'authors', 'keywords',
        'meta_keywords', 'meta_description', 'tags', 'summary', 'source', 'label'
    ]

    # Writes each article to a csv-file. The meta-data such as "headline", "author" and "text" is added to the relevent columns,
    # The type is is set to reliable for all the articles. 
    # The label is set to 1 for all articles
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(headers)  

        for article in articles:
            row = [''] * len(headers)
            row[headers.index('title')] = article.get('Headline', '')
            row[headers.index('authors')] = article.get('Author', '')
            row[headers.index('content')] = article.get('Text', '')
            row[headers.index('type')] = 'reliable'                     # Set type to "reliable"
            row[headers.index('label')] = 1                             # Set label to 1
            writer.writerow(row)

# Calls the function, with the input file and the output file as arguments
convert_to_csv('/Users/simonhvidtfeldt/Group_project/output_big.txt',
                '/Users/simonhvidtfeldt/Group_project/output_big_matched.csv')

Now we have converted the scraped article from bbc into the same format. Furthermore we have also added their type, which is reliable, and set their label to 1. 

In [21]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define file path and chunk size
file_path = "/Users/simonhvidtfeldt/Group_project/output_big_matched.csv"
chunksize = 25000

# Define your cleaning function
def clean_text(data):
    if not isinstance(data, str):  # Handle NaN values safely
        return ""
    data = data.lower()
    data = re.sub(r'\s+', " ", data)
    data = re.sub(r'\d{1,2}[./-]\d{1,2}[./-]\d{2,4}', "<DATE>", data)
    data = re.sub(r'(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec).? \d{1,2},? \d{4}', "<DATE>", data)
    data = re.sub(r'\d{4}-\d{2}-\d{2}', "<DATE>", data)
    data = re.sub(r'[\w._%+-]+@[\w.-]+\.[a-zA-Z]{2,}', "<EMAIL>", data)
    data = re.sub(r'http[s]?://[^\s]+', "<URL>", data)
    data = re.sub(r'\d+(\.\d+)?', "<NUM>", data)
    return data

# Initialize stopwords and stemmer
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

# Define function to tokenize, remove stopwords, and stem
def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    filtered_tokens = [ps.stem(word) for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

# List of columns to process
columns_to_clean = ["Unamed: 0",
    "id", "domain", "type", "url", "content", "scraped_at", "inserted_at",
    "updated_at", "title", "authors", "keywords", "meta_keywords",
    "meta_description", "tags", "summary"
]

#Process and store all chunks in a single DataFrame
preprocessed_data = pd.DataFrame()

for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
    for col in columns_to_clean:
        if col in chunk.columns:
            # Clean
            chunk[col] = chunk[col].apply(clean_text)
            #Remove stopwords
            chunk[col] = chunk[col].astype(str).apply(
                lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words])
            )
            #Tokenize and stem
            chunk[col] = chunk[col].apply(tokenize_and_stem)
    # Append chunk to final DataFrame
    preprocessed_data = pd.concat([preprocessed_data, chunk], ignore_index=True)

# Save the final DataFrame to a CSV file
preprocessed_data.to_csv("output_big_matched_cleaned.csv", index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonhvidtfeldt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simonhvidtfeldt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now, that we have cleaned the file we can add it to the 995.000 rows. 

In [22]:
import csv
import sys

def add_csv_files(file1, file2, output_file):
    # Increase the field size limit
    max_int = sys.maxsize
    while True:
        try:
            csv.field_size_limit(max_int)
            break
        except OverflowError:
            max_int = int(max_int / 10)

    # Read the first CSV file
    with open(file1, 'r') as f1:
        reader = csv.reader(f1)
        header = next(reader)  # Read the header
        rows_file1 = list(reader)  # Read all rows

    # Read the second CSV file
    with open(file2, 'r') as f2:
        reader = csv.reader(f2)
        next(reader)  # Skip the header since it is the same as file1
        rows_file2 = list(reader)  # Read all rows

    # Combine rows from both files
    combined_rows = rows_file1 + rows_file2

    # Write the combined rows to the output file
    with open(output_file, 'w', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(header)  # Write the header
        writer.writerows(combined_rows)  # Write all rows

# Example usage
add_csv_files('/Users/simonhvidtfeldt/Group_project/cleaned_file_with_label.csv',           # file1
                  "/Users/simonhvidtfeldt/Group_project/output_big_matched_cleaned.csv",    # file2
                    '/Users/simonhvidtfeldt/Group_project/scraped_plus_995k.csv')           # The file we want to created, where they have been put together 

In [24]:
import pandas as pd

# File paths
file1 = "/Users/simonhvidtfeldt/Group_project/cleaned_file_with_label.csv"
file2 = "/Users/simonhvidtfeldt/Group_project/scraped_plus_995k.csv"

# Load the CSV files and count rows
def count_rows(file_path):
    try:
        df = pd.read_csv(file_path)
        return len(df)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return 0

# Count rows in both files
rows_file1 = count_rows(file1)
rows_file2 = count_rows(file2)

# Calculate the difference
difference = abs(rows_file1 - rows_file2)

# Print results
print(f"Number of rows in cleaned_file_with_label.csv: {rows_file1}")
print(f"Number of rows in scraped_plus_995k.csv: {rows_file2}")
print(f"Difference in rows: {difference}")

Number of rows in cleaned_file_with_label.csv: 678609
Number of rows in scraped_plus_995k.csv: 679407
Difference in rows: 798


As can be seen by, we have succesfully added the additional 798 articles to the 995k rows. Great!
now lets try to run the combined file through the simple logistic regression.

In [25]:
"""""""""""""""""""""""""""""""""""""""""""""
CODE FROM PART_2_TASK_1 *EDITED*

LOGISTIC REGRESSION FOR 995k + scraped 
"""""""""""""""""""""""""""""""""""""""""""""

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np

print("Beginning Logistic regression for 995k + scraped:")

# Load CSV
data = pd.read_csv("/Users/simonhvidtfeldt/Group_project/scraped_plus_995k.csv")
print("  - Loaded CSV.")

# Ensure text data is string type
data["content"] = data["content"].fillna("").astype(str)
print("  - Ensured text data is string type.")

# Vectorize text data
vectorizer = TfidfVectorizer(max_features=10000, binary=True)
X = vectorizer.fit_transform(data["content"])
print("  - Vectorized text data.")

# Use the binary 'label' column as the target variable
y = data["label"].astype(int)  # Ensure 'label' is integer type
print("  - Used the binary 'label' column as the target variable.")

# Splitting dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print("  - Splitting dataset into train, validation, and test sets.")

# Implementing logistic regression
clf = LogisticRegression(max_iter=500, random_state=42, solver="saga", n_jobs=-1)
clf.fit(X_train, y_train)
print("  - Implementing logistic regression.")

# Evaluating the model
y_pred = clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average="binary")  # Use 'binary' for binary classification

# Display results
print("\nResults of logistic regression of 995k + scraped:")
print(f"  - F1-score: {f1:.4f}\n")
print("Hyperparameters for Logistic Regression:")
print(f"  - Max Iterations: {clf.max_iter}")
print(f"  - Solver: {clf.solver}")
print(f"  - Random State: {clf.random_state}")


Beginning Logistic regression for 995k + scraped:
  - Loaded CSV.
  - Ensured text data is string type.
  - Vectorized text data.
  - Used the binary 'label' column as the target variable.
  - Splitting dataset into train, validation, and test sets.
  - Implementing logistic regression.

Results of logistic regression of 995k + scraped:
  - F1-score: 0.9198

Hyperparameters for Logistic Regression:
  - Max Iterations: 500
  - Solver: saga
  - Random State: 42


For part 2, task 1 we got an F1 score of 0.9194. By adding the extra 799 articles we were able to increase that score to 0.9198. Lets calculate the increase ine percentage. 

*Subtract the old F1 score from the new F1 score:*

$$0.9198 - 0.9194 = 0.0004$$

*Divide the difference by the old F1 score:*

$$\frac{0.0004}{0.9194}≈0.000435$$

*Multiply by 100 to get the percentage increase:*

$$0.000435 \cdot 100 = 0.0435\%$$

So, the F1 score increased by 0.0435%. This is a very small increase. Either, the extra articles didn't provide significant new information, or the model is already very good at classifying the articles.